# Problem 2. More IV in Practice and the Bootstrap

In [15]:
#import required packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy as sc
import matplotlib as plt
import sys 
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [16]:
#set notebook parameters to show data in pandas frames
pd.options.display.max_columns = 10
pd.options.display.max_rows = 5

#establish random seed
np.random.seed(1)

In [17]:
#grab the dataframe
df=pd.read_csv("lottery_newstata.csv")
df

,female,year,lotcateg,z,d,lnw
0,0,1988,4,0,1,2.751068
1,1,1989,5,1,1,4.365279
...,...,...,...,...,...,...
1474,1,1989,5,1,1,2.857421
1475,0,1989,6,0,0,2.603315


# 1. Write the loop that estimates the LATE parameter by year and category

Recall the definition of LATE. LATE:= DIFFERENCE OF (average potential outcome for compliers when they DO receive treatment) AND ( average potential ouctome for compliers when they DONT receive treatment). We obtain the first part from the datadirectly,the trickly part is to capture average potential outcome for compliers when they dont receive treatment. This is because those who don't receive treatment in colnrol group could be compliers or never takers. <br><br>
 ## <span style="color:green">As I mentioned on my note on problem 1, this is difficult to visualize. Treatment means the lottery (Z==1),control Z==0, and outcome means D? SO the government runs admissions lottery and tells parents to not entroll if Z==0? So compliers will be D==0 and so will never takers.   How iss Z==0 and D==1 poissible? Like, if admissions are a must to attend school, how D==1 if Z==0 is possible? </span>

1. We need to assume that our dataset does not contain  always takers.

2. Average outcome in control group equals:= (Average outcome of compliers when they DONT receive treatment)*proportion of compliers + (Average outcome for never takers when they dont receive treatment) * Proportion of never takers. 

We can solve for "Average outcome of compliers when they DONT receive treatment" and calculate LATE, because all other variables are a function of data. 

##  <span style="color:blue"> Below is just an example of calculations for the entire dataset - we will iterate these in a loop </span>

In [18]:
#Get probability of compliers and never takers 
P_COMPLIERS = df[["z","d"]][df["z"]==1]["d"].mean()
P_NEVER_TAKER=1- P_COMPLIERS
P_NEVER_TAKER, P_COMPLIERS

(0.06960784313725488, 0.9303921568627451)

In [19]:
#Outcome for compliers when they receive treatment
COMPLIER_OUTCOME_TREATED = df[["z","d","lnw"]][df["z"]==1][df["d"]==1]["lnw"].mean()
COMPLIER_OUTCOME_TREATED

3.1978332297154908

In [20]:
# Estimate outcome for never takers in control group - obtain from the treatment group Z=1, d=0
NEVER_TAKER_OUTCOME=df[["z","d","lnw"]][df["z"]==1][df["d"]==0]["lnw"].mean()
NEVER_TAKER_OUTCOME

3.0092357887323935

In [21]:
#grab average outcome in control grop
AOCG = df[df["z"]==0]["lnw"].mean()
AOCG

3.0873473201754384

In [22]:
#obtain missing ingredient to get LATE
COMPLIER_OUTCOME_NOT_TREATED = (AOCG- NEVER_TAKER_OUTCOME*P_NEVER_TAKER)/P_COMPLIERS
COMPLIER_OUTCOME_NOT_TREATED

3.0931912809051076

In [23]:
#finally, calculate LATE
LATE = COMPLIER_OUTCOME_TREATED-COMPLIER_OUTCOME_NOT_TREATED
LATE

0.10464194881038313

##  <span style="color:blue"> Now run throught he loop similar to the one in problem 1 </span>

In [24]:
#get list of categories
categs=df["lotcateg"].unique()
categs

array([4, 5, 6, 3])

In [25]:
#get list of years
years = df["year"].unique()
years

array([1988, 1989])

In [26]:
#create a new dataframe where will sum the results
summary =pd.DataFrame(index=["Category", "Year","LATE"])

#iterate/#loop over the list, run regression, extract beta value for Z and t-statistic, append to the dataframe 
for year in years:
    for cat in categs:
        
        #generate the subset based on year and category
        subset = df[df["year"]==year][df["lotcateg"]==cat]

        
        #Get probability of compliers and never takers 
        P_COMPLIERS = subset[["z","d"]][subset["z"]==1]["d"].mean()
        P_NEVER_TAKER=1- P_COMPLIERS
        
        #Outcome for compliers when they receive treatment
        COMPLIER_OUTCOME_TREATED = subset[["z","d","lnw"]][subset["z"]==1][subset["d"]==1]["lnw"].mean()
        
        # Estimate outcome for never takers in control group - obtain from the treatment group Z=1, d=0
        NEVER_TAKER_OUTCOME=subset[["z","d","lnw"]][subset["z"]==1][subset["d"]==0]["lnw"].mean()
        
        #grab average outcome in control grop
        AOCG = subset[subset["z"]==0]["lnw"].mean()
        
        #obtain missing ingredient to get LATE
        COMPLIER_OUTCOME_NOT_TREATED = (AOCG- NEVER_TAKER_OUTCOME*P_NEVER_TAKER)/P_COMPLIERS
        
        #finally, calculate LATE
        LATE = COMPLIER_OUTCOME_TREATED-COMPLIER_OUTCOME_NOT_TREATED

        #shove them all into one list
        add = [str(cat),str(year), LATE]

        #get new column name in dataset to append the result
        col = "LATE_" + str(cat) + "_" + str(year)

        #now transpose and add the list as a column
        summary[col]= add
summary

,LATE_4_1988,LATE_5_1988,LATE_6_1988,LATE_3_1988,LATE_4_1989,LATE_5_1989,LATE_6_1989,LATE_3_1989
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989
LATE,0.0450943,0.101,0.147633,-0.143376,0.00342996,0.179965,0.104921,0.527623


## <span style="color:blue"> DONE! </span>

## 2. Aggregate the year-category-wise LATE parameter estimates using a weighted average with the appropriate weights.

Appropriate weights are the number of observations in this cluster ( category*year ) in the dataset 

In [27]:
#get the counts for cartesian product of year*cat
weights = df[["year","lotcateg","z"]].groupby(["year","lotcateg"]).count()
weights

z
year lotcateg     
1988 3          84
     4         209
...            ...
1989 5         210
     6         247

[8 rows x 1 columns]

In [28]:
#create weights column with appropriate weights summing up to 1
weights["weight"] = weights["z"]/df.shape[0]
weights.drop("z", axis=1, inplace=True)
weights=weights.reset_index()
weights

,year,lotcateg,weight
0,1988,3,0.056911
1,1988,4,0.141599
...,...,...,...
6,1989,5,0.142276
7,1989,6,0.167344


In [29]:
#prepare the dataset with LATE calculations from #1 by transposing and housekeeping
late=summary.transpose().sort_values(["Year","Category"])
late.Category=late.Category.astype(int)
late.Year = late.Year.astype(int)
late

,Category,Year,LATE
LATE_3_1988,3,1988,-0.143376
LATE_4_1988,4,1988,0.0450943
...,...,...,...
LATE_5_1989,5,1989,0.179965
LATE_6_1989,6,1989,0.104921


In [30]:
#merge weights with LATEs
merged = pd.merge(late,weights, left_on=["Category","Year"],right_on=["lotcateg","year"])
merged.drop(["year","lotcateg"], axis=1, inplace=True)
merged

,Category,Year,LATE,weight
0,3,1988,-0.143376,0.056911
1,4,1988,0.0450943,0.141599
...,...,...,...,...
6,5,1989,0.179965,0.142276
7,6,1989,0.104921,0.167344


In [31]:
#calculate weighted LATE for the entire datset:
np.sum(merged.LATE*merged.weight)

0.10967956853429342

## <span style="color:blue"> DONE! </span>

## 3. Provide a 90% conﬁdence interval for the weighted-average of the LATE using the bootstrap (justify your clustering). Set your seed to 1 and use 1,000 bootstrap resamples.

##  <span style="color:blue"> To accomplish this, use random choice function and specify weights in the parameters accordingly </span>

In [32]:
#load the required package
import random


In [33]:
#initiate empty list - we will be inputing sample means there to get 1000 bootstraps
lates = []

#set seed to 1
np.random.seed(1)

#init for loop
for i in range(1000):
    
    #this line tells that we sample from LATE columns with probability weighted by weights column
    entry = np.mean(np.random.choice(merged.LATE,size=100, replace=True, p=merged.weight))
    
    # add average to the list
    lates.append(entry)
    
    
#what's the ;length of the list
len(lates)

1000

In [34]:
#sort the list, order does not matter
lates.sort(reverse=True)

#now remove first 50 and last 50 entries. 
lates=lates[50:-50]

#confirm we have 900 entries left
len(lates)

900

In [35]:
#the max of the new list will be the upper bound of the confidence interval
max(lates)

0.13150634425493185

In [36]:
#the min of the new list will be the lower bound of the confidence interval
min(lates)

0.08903892547211698

## <span style="color:blue"> Our condifence interval is below</span>

In [37]:
print((min(lates),max(lates)))

(0.08903892547211698, 0.13150634425493185)


## <span style="color:blue"> We observe that our value falls within the confidence interval </span>

## <span style="color:blue"> DONE! </span>